In [1]:
def rgb_or_hsv():
    user_input=input("Enter 'R' to use RGB or 'H' to use HSV: ")

    if user_input.upper() == 'H':
        return 0
    elif user_input.upper() == 'R':
        return 1
    
    return -1

def get_color_values(user_input):
                    # Lower     # Upper
    color_values = [[[240,95,90],[246,244,253]],       # HSV
                   [[6,6,214], [120,120,255]]]   # RGB
    return color_values[user_input]

def get_color_scale(user_input):
    color_scale = [cv2.COLOR_BGR2HSV, cv2.COLOR_BGR2RGB]
    return color_scale[user_input]

def resize(frame):
    return cv2.resize(frame, (800, 600))

In [ ]:
import cv2
import numpy as np

user_input = rgb_or_hsv()
color_values = get_color_values(user_input)
color_scale = get_color_scale(user_input)
webcam = cv2.VideoCapture(0, cv2.CAP_DSHOW)

while True: 
    # Reading the video from the webcam in image frames 
    _, imageFrame = webcam.read() 

    # Convert the imageFrame in BGR to HSV color space
    hsvFrame = cv2.cvtColor(imageFrame, color_scale)
  
    # Set range for blue color and define mask
    blue_lower = np.array(color_values[0])
    blue_upper = np.array(color_values[1])
    blue_mask = cv2.inRange(hsvFrame, blue_lower, blue_upper) 

    # Morphological Transform, Dilation for blue color 
    kernel = np.ones((5, 5), "uint8")
    blue_mask = cv2.dilate(blue_mask, kernel) 
    res_blue = cv2.bitwise_and(imageFrame, imageFrame, mask=blue_mask)   

    # Creating contour to track blue color 
    contours, hierarchy = cv2.findContours(blue_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
    for contour in contours: 
        area = cv2.contourArea(contour) 
        if area > 300: 
            x, y, w, h = cv2.boundingRect(contour) 
            imageFrame = cv2.rectangle(imageFrame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Display the frame with detected blue color
    cv2.imshow("Multiple Color Detection in Real-Time", imageFrame)
    cv2.imshow('Isolating Objects with Specified Color',resize(res_blue))

    # Program Termination 
    if cv2.waitKey(10) & 0xFF == ord('q'): 
        webcam.release() 
        cv2.destroyAllWindows() 
        break
